# Food Recommendation Systems 
Almost all of us have heard about the phrase "prevention is better than cure" and there is no better prevention than the right type of foods. The ingredients in food have medicinal properties and a recommendation system based on those ingredients would be useful for many like me


In [1]:
%env PYDEVD_DISABLE_FILE_VALIDATION=1


env: PYDEVD_DISABLE_FILE_VALIDATION=1


In [2]:
import pandas as pd
import re

## The Interactions between Users and the recipes

The raw_interaction.csv has the user_id and recipe_id and the ratings given by the users. Get the full dataset from kaggle if needed https://www.kaggle.com/code/aswathchandrasekar/food-recommendation-system

In [3]:
filtered_user_interactions=pd.read_csv("filtered_users.csv")
filtered_user_interactions.head()

,user_id,recipe_id,date,rating,review
0,33188,58224,2004-07-30,5,These are so good!! My family devoured them - ...
1,33443,58224,2004-03-21,5,"This was easy, delicious, and extremely versat..."
2,28346,58224,2004-07-11,5,Yummm!! I sampled this a few times as it was ...
3,115053,58224,2004-08-18,4,This was great. I halved the recipe for just ...
4,101823,58224,2004-09-25,4,This was a cheesy twist on sloppy joes. It ma...


## Recipes dataset
<p>The recipes dataset has attributes such as ingredients, steps which can be used for developing a content-based collaborative filtering recommendation system.</p>

In [4]:
recipes_df=pd.read_csv("300_recipes.csv")
recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
recipes_df.describe()

,id,minutes,contributor_id,n_steps,n_ingredients
count,300.000000,300.000000,3.000000e+02,300.000000,300.000000
mean,155206.533333,212.570000,6.895001e+06,9.393333,9.406667
std,121813.667210,1441.409589,1.154814e+08,5.809572,4.220664
min,190.000000,0.000000,1.533000e+03,1.000000,2.000000
25%,53754.500000,25.000000,4.170600e+04,5.000000,6.000000
50%,112971.500000,45.000000,9.500750e+04,8.000000,9.000000
75%,235425.750000,80.000000,2.958955e+05,12.000000,12.000000
max,523359.000000,20160.000000,2.000415e+09,41.000000,23.000000


## Medicines dataset
The medicine dataset has the medicinal benefits for common ingredients like Garlic, ginger etc

In [6]:
medicinal_df=pd.read_csv("medicines.csv").dropna()
medicinal_df.head(10)

,Ingredients,benefits
0,Sandy everlasting,Gastrointestinal disorders
1,Diuretic herbal tea combinations,Urinary tract and genital disorders
2,Milkthistle Fruit,Gastrointestinal disorders
3,Garlic,Cough and cold
4,Garlic,Circulatory disorders
5,"Mastic (Mastix, Pistaciae lentisci resina)",Skin disorders and minor wounds
6,"Mastic (Mastix, Pistaciae lentisci resina)",Gastrointestinal disorders
7,Wild Strawberry Leaf,Urinary tract and genital disorders
8,Wild Strawberry Leaf,Gastrointestinal disorders
9,Mallow leaf,Cough and cold


In [7]:
medicinal_df.describe()  # There are 280 ingredients in the dataset

,Ingredients,benefits
count,280,280
unique,174,14
top,Yarrow,Gastrointestinal disorders
freq,4,55


## Matching the medicinal properties
Using the medicinal benefits of ingredients in the medicinal dataset, we match it to the recipes dataset by tokenizing and matching the words and saving it to a separate csv

In [8]:

from fuzzywuzzy import fuzz

# Create a new column for medicinal properties
recipes_df['Medicinal_Properties'] = ''

# Create an empty list to store the medicinal properties
medicinal_properties = []

# Convert the 'ingredients' column from string to list
recipes_df['ingredients'] = recipes_df['ingredients'].str.split(',')
print("starting")
# Iterate through the recipe ingredients
for ingredients in recipes_df['ingredients']:
    found_ingredients = []  # List to store found ingredients in the current recipe

    # Check if ingredients is not NaN or None
    if isinstance(ingredients, list):
        # Iterate through each ingredient in the recipe
        for ingredient in ingredients:
            best_match_score = 0
            best_match_ingredient = None

            # Find the best matching ingredient in the medicinal DataFrame
            for medicinal_ingredient in medicinal_df['Ingredients']:
                match_score = fuzz.token_set_ratio(ingredient.strip().lower(), medicinal_ingredient.strip().lower())
                if match_score > best_match_score:
                    best_match_score = match_score
                    best_match_ingredient = medicinal_ingredient

            # Check if a matching ingredient with sufficient similarity score was found
            if best_match_ingredient and best_match_score >= 80:
           
                found_ingredients.append(best_match_ingredient)

    # Get the medicinal properties for the found ingredients
    properties = []
    for found_ingredient in found_ingredients:
    
        properties.extend(medicinal_df.loc[medicinal_df['Ingredients'] == found_ingredient, 'benefits'].values)
    medicinal_properties.append(properties)

# Assign the medicinal properties to the new column
recipes_df['Medicinal_Properties'] = medicinal_properties
print("done ")

recipes_df.to_csv('recipes_with_medicinal_properties.csv', index=False)


starting
done 


In [9]:
recipes_df=pd.read_csv("recipes_with_medicinal_properties.csv")
recipes_df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,Medicinal_Properties
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"[""['winter squash'"", "" 'mexican seasoning'"", ""...",7,[]
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"[""['prepared pizza crust'"", "" 'sausage patty'""...",6,[]
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"[""['ground beef'"", "" 'yellow onions'"", "" 'dice...",13,[]
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","[""['spreadable cheese with garlic and herbs'"",...",11,"['Cough and cold', 'Circulatory disorders']"
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"[""['tomato juice'"", "" 'apple cider vinegar'"", ...",8,"['Mouth and throat disorders', 'Gastrointestin..."


In [10]:
recipes_df=recipes_df[recipes_df.astype(str)['Medicinal_Properties'] != '[]'] #drop empty lists row in medicinal_properties
recipes_df["name"]

3                           alouette  potatoes
4           amish  tomato ketchup  for canning
6                        aww  marinated olives
7               backyard style  barbecued ribs
10                 berry  good sandwich spread
                        ...                   
292                 this old house    dressing
293                             trini  burgers
294    we hate zucchini bread   zucchini bread
295                             white  sangria
297          windy s  sweet and sour meatballs
Name: name, Length: 158, dtype: object

# Content Based Recommendation System
We will use TF-IDF (Term Frequency - Inverse Document Frequency) to see how important the words are in a recipe and then we will use cosine-similarity to compare the similarity between teh recipes

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import scipy.sparse


#c
# Combine ingredients, steps, and medicinal properties into a single text column
recipes_df['text'] = recipes_df['ingredients'] + ' ' + recipes_df['steps'] + ' ' + recipes_df['Medicinal_Properties'].apply(lambda x: ' '.join(x))
# Check for missing values in the 'text' column
missing_values = recipes_df['text'].isnull().sum()
print("Missing values:", missing_values)

# Drop rows with missing values in the 'text' column
recipes_df = recipes_df.dropna(subset=['text'])

# Check for empty strings in the 'text' column
empty_strings = (recipes_df['text'] == '').sum()
print("Empty strings:", empty_strings)
# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Compute the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform(recipes_df['text'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

# Print the cosine similarity matrix
print(cosine_sim)
print(len(cosine_sim))

Missing values: 0
Empty strings: 0
[[1.         0.07213552 0.09945863 ... 0.10555605 0.05475662 0.06869464]
 [0.07213552 1.         0.04892975 ... 0.08067661 0.04456903 0.08760838]
 [0.09945863 0.04892975 1.         ... 0.0951513  0.04507138 0.02968579]
 ...
 [0.10555605 0.08067661 0.0951513  ... 1.         0.05086896 0.07766115]
 [0.05475662 0.04456903 0.04507138 ... 0.05086896 1.         0.0287531 ]
 [0.06869464 0.08760838 0.02968579 ... 0.07766115 0.0287531  1.        ]]
158


In [12]:
recipes_df.reset_index(drop=True, inplace=True)


## Getting recommendations
Now we are recommending the top 5 recipes which are similar when we pass in a recipe title

In [13]:

# Function to get recipe recommendations based on cosine similarity
def get_recipe_recommendations(recipe_title, cosine_sim_matrix, recipes_df, top_n=5):
    # Get the index of the recipe title
    recipe_index = recipes_df[recipes_df['name'] == recipe_title].index[0]

    # Get the cosine similarity scores of the recipe index
    similarity_scores = cosine_sim_matrix[recipe_index]


    # Sort the recipes based on similarity scores
    top_indices = np.argsort(similarity_scores)[::-1][1:top_n+1]

    # Get the recipe IDs of top similar recipes
    top_recipe_ids = recipes_df['id'].iloc[top_indices]

    # Return the top recommended recipe IDs
    #return top_recipe_ids
    return [(recipes_df.loc[index, 'id'], recipes_df.loc[index, 'name']) for index in top_indices]

# Example usage
recipe_title = 'amish  tomato ketchup  for canning'
recommendations = get_recipe_recommendations(recipe_title, cosine_sim, recipes_df)
print(f"Recommended recipes for '{recipe_title}':")
_ = [print(f"{i}: {recipe_name}") for i, (_, recipe_name) in enumerate(recommendations, 1)]


Recommended recipes for 'amish  tomato ketchup  for canning':
1: momma s special  marinade
2: chef salad  dressing
3: fire  sauce
4: better then bush s  baked beans
5: i hate miracle whip  but i love this potato salad that s


In [14]:
row_1 = recipes_df.loc[recipes_df['id'] == 59389]
medicinal_properties = row_1['Medicinal_Properties'].values[0]
print(medicinal_properties)


['Cough and cold', 'Circulatory disorders']


## Recommending based on medicinal properties

In [15]:
def search_recipes_by_medicinal_properties(medicinal_property, recipes_df, top_n=5):
    # Define the regular expression pattern for matching
    pattern = r'\b{}\b'.format(medicinal_property)

    # Filter recipes by the given medicinal property using regular expression matching
    filtered_recipes = recipes_df[recipes_df['Medicinal_Properties'].str.contains(pattern, case=False, regex=True)]

    # If there are no recipes with the given medicinal property, return an empty list
    if len(filtered_recipes) == 0:
        return []

    # Get a random recipe from the filtered recipes
    random_recipe = filtered_recipes.sample(n=1)

    # Get recommendations based on the random recipe
    recommendations = get_recipe_recommendations(random_recipe['name'].values[0], cosine_sim, recipes_df, top_n)

    # Return the recommended recipe IDs and titles
    return [(recipe_id, recipe_title) for recipe_id, recipe_title in recommendations]

# Example usage
medicinal_property = 'Circulatory disorders'
recommendations = search_recipes_by_medicinal_properties(medicinal_property, recipes_df)
print(f"Recommended recipes for '{medicinal_property}':")
_ = [print(f"{i}: {recipe_title}") for i, (_, recipe_title) in enumerate(recommendations, 1)]



Recommended recipes for 'Circulatory disorders':
1: 50 chili   for the crockpot
2: fool the meat eaters  chili
3: slightly spicy  black bean burgers
4: outback   steak rub
5: forgotten  minestrone


# User-based Colloborative System

In [16]:
filtered_user_interactions.head(5)

,user_id,recipe_id,date,rating,review
0,33188,58224,2004-07-30,5,These are so good!! My family devoured them - ...
1,33443,58224,2004-03-21,5,"This was easy, delicious, and extremely versat..."
2,28346,58224,2004-07-11,5,Yummm!! I sampled this a few times as it was ...
3,115053,58224,2004-08-18,4,This was great. I halved the recipe for just ...
4,101823,58224,2004-09-25,4,This was a cheesy twist on sloppy joes. It ma...


## Filtering recipes based on ratings
We only keep the recipes that has atleast more than 5 ratings

In [17]:
agg_ratings = filtered_user_interactions.groupby('recipe_id').agg(mean_rating = ('rating', 'mean'),
                                                number_of_ratings = ('rating', 'count')).reset_index()
# Keep the recipes with over 5 ratings
agg_ratings_GT100 = agg_ratings[agg_ratings['number_of_ratings']>5]
agg_ratings_GT100.head()

,recipe_id,mean_rating,number_of_ratings
2,17265,4.133333,30
3,18537,3.888889,9
6,22123,4.500000,6
7,22526,4.549020,51
10,24514,4.222222,9


In [18]:
# Check popular recipes
agg_ratings_GT100.sort_values(by='number_of_ratings', ascending=False).head()

,recipe_id,mean_rating,number_of_ratings
93,155959,4.362832,113
82,125195,4.622642,53
62,83950,3.962264,53
7,22526,4.549020,51
71,95534,4.707317,41


In [19]:
df_GT100 = pd.merge(filtered_user_interactions, agg_ratings_GT100[['recipe_id']], on='recipe_id', how='inner')
df_GT100.head()

,user_id,recipe_id,date,rating,review
0,33188,58224,2004-07-30,5,These are so good!! My family devoured them - ...
1,33443,58224,2004-03-21,5,"This was easy, delicious, and extremely versat..."
2,28346,58224,2004-07-11,5,Yummm!! I sampled this a few times as it was ...
3,115053,58224,2004-08-18,4,This was great. I halved the recipe for just ...
4,101823,58224,2004-09-25,4,This was a cheesy twist on sloppy joes. It ma...


In [20]:
# Number of users
print('The ratings dataset has', df_GT100['user_id'].nunique(), 'unique users')
# Number of movies
print('The ratings dataset has', df_GT100['recipe_id'].nunique(), 'unique recipes')
# Number of ratings
print('The ratings dataset has', df_GT100['rating'].nunique(), 'unique ratings')
# List of unique ratings
print('The unique ratings are', sorted(df_GT100['rating'].unique()))


The ratings dataset has 725 unique users
The ratings dataset has 37 unique recipes
The ratings dataset has 6 unique ratings
The unique ratings are [0, 1, 2, 3, 4, 5]


In [21]:
# Create the user-item rating matrix
# Set 'user_id' as the index

user_ratings = df_GT100.pivot(index='user_id', columns='recipe_id', values='rating')
user_ratings.fillna(0, inplace=True)

user_ratings.head(5)



recipe_id,17265,18537,22123,22526,24514,26835,29214,30473,31835,43026,...,155959,161370,164291,166089,197001,207525,219681,220186,270259,353171
user_id,,,,,,,,,,,,,,,,,,,,,
4470,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6258,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6512,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
user_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 725 entries, 4470 to 2002319670
Data columns (total 37 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   17265   725 non-null    float64
 1   18537   725 non-null    float64
 2   22123   725 non-null    float64
 3   22526   725 non-null    float64
 4   24514   725 non-null    float64
 5   26835   725 non-null    float64
 6   29214   725 non-null    float64
 7   30473   725 non-null    float64
 8   31835   725 non-null    float64
 9   43026   725 non-null    float64
 10  52804   725 non-null    float64
 11  58224   725 non-null    float64
 12  63986   725 non-null    float64
 13  67547   725 non-null    float64
 14  70005   725 non-null    float64
 15  83133   725 non-null    float64
 16  83950   725 non-null    float64
 17  84797   725 non-null    float64
 18  90921   725 non-null    float64
 19  95534   725 non-null    float64
 20  108804  725 non-null    float64
 21  111462  725 non-null    float64
 2

## Recommendations based on cosine similarity
We are creating a user matrix using cosine similarity to find out similar users and get the most similar users based on the weighted ratings 

In [23]:
user_sim = cosine_similarity(user_ratings)
user_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [24]:
def get_top_similar_users(user_id, user_sim_matrix, user_ratings, top_n=5):
    # Get the index of the user_id in the user_ratings DataFrame
    user_index = user_ratings.index.get_loc(user_id)

    # Get the cosine similarity scores of the user index
    similarity_scores = user_sim_matrix[user_index]

    # Sort the users based on similarity scores
    user_scores = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)

    # Get the indices of top similar users
    top_user_indices = [i[0] for i in user_scores[1:top_n+1]]

    # Get the user ids corresponding to the top similar user indices
    top_user_ids = [user_ratings.index[i] for i in top_user_indices]

    return top_user_ids
users=get_top_similar_users(4470, user_sim, user_ratings, top_n=5)
users

[18391, 26478, 28671, 37471, 75497]

In [25]:
def get_user_recipe_recommendations(user_id, user_ratings, user_sim_matrix, recipes_df, top_n=5):
    # Create a dictionary to map user IDs to indices
    user_id_to_index = {user_id: index for index, user_id in enumerate(user_ratings.index)}
    # Get the index of the user_id in the user_ratings DataFrame
    user_index = user_id_to_index[user_id]
    # Get the top similar users for the given user
    top_similar_users = get_top_similar_users(user_id, user_sim_matrix, user_ratings, top_n)
    # Get the recipes rated by the top similar users
    similar_user_ratings = user_ratings.loc[top_similar_users]
    top_similar_user_indices = [user_id_to_index[user_id] for user_id in users]
    # Compute the weighted average ratings for the recipes
    weighted_ratings = np.dot(similar_user_ratings.values.T, user_sim_matrix[user_index, top_similar_user_indices]) / np.sum(user_sim_matrix[user_index, top_similar_user_indices])
    # Get the top 5 recipe IDs
    top_recipe_indices = np.argsort(weighted_ratings)[::-1][:top_n]  
    recommendations = recipes_df.iloc[top_recipe_indices]
    return recommendations

recommendations=get_user_recipe_recommendations(4470, user_ratings, user_sim, recipes_df, top_n=5)


# Print the top 5 recipe names with numbering
print("The top suggestions for you are:")
for i, recipe_name in enumerate(recommendations['name'], 1):
    print(f"{i}: {recipe_name}")


The top suggestions for you are:
1: amish  tomato ketchup  for canning
2: crispy crunchy  chicken
3: love is in the air  beef fondue   sauces
4: chile rellenos
5: emotional balance  spice mixture


The problem with many user based recommendation system is that they often suggest the most rated items on top but a user normally wants to find new suggestions that he doesn't know about so we need to randomly suggest some recipes

In [26]:
def get_user_recipe_recommendations(user_id, user_ratings, user_sim_matrix, recipes_df, top_n=2, random_n=3):
    # Create a dictionary to map user IDs to indices
    user_id_to_index = {user_id: index for index, user_id in enumerate(user_ratings.index)}

    # Get the index of the user_id in the user_ratings DataFrame
    user_index = user_id_to_index[user_id]

    # Get the top similar users for the given user
    top_similar_users = get_top_similar_users(user_id, user_sim_matrix, user_ratings,top_n)

    # Get the recipes rated by the top similar users
    similar_user_ratings = user_ratings.loc[user_ratings.index.isin(top_similar_users)]

    # Get the indices of the top similar users
    top_similar_user_indices = [user_id_to_index[user_id] for user_id in top_similar_users]

    if len(top_similar_users) >= top_n:
        # Compute the weighted average ratings for the recipes
        weighted_ratings = np.dot(similar_user_ratings.values.T, user_sim_matrix[user_index, top_similar_user_indices]) / np.sum(user_sim_matrix[user_index, top_similar_user_indices])
        # Sort the recipes based on the weighted ratings
        sorted_recipe_indices = np.argsort(weighted_ratings)[::-1][:top_n]
    else:
        # If there are fewer top similar users than top_n, return all available recommendations
        sorted_recipe_indices = np.arange(len(recipes_df))

    # Get the top rated recommendations
    top_recommendations = recipes_df.iloc[sorted_recipe_indices]

    # Get some random recommendations
    random_recommendations = recipes_df.sample(n=random_n)

    # Concatenate the top rated and random recommendations
    recommendations = pd.concat([top_recommendations, random_recommendations])

    return recommendations


recommendations=get_user_recipe_recommendations(4470, user_ratings, user_sim, recipes_df, top_n=2, random_n=3)
print("The top suggestions for you are:")
for i, recipe_name in enumerate(recommendations['name'], 1):
    print(f"{i}: {recipe_name}")

The top suggestions for you are:
1: amish  tomato ketchup  for canning
2: love is in the air  beef fondue   sauces
3: lite  cretons
4: forgotten  minestrone
5: splenda   d  cheesecake sugar free low carb


# Hybrid Colloborative System
It is often better to go with the best of both worlds and a hybrid collobarativse system with both user based and item to item collobarative system would be ideal for a food recommendation system

In [27]:
def get_combined_recommendations(user_id, recipe_title, user_ratings, user_sim_matrix, cosine_sim_matrix, recipes_df, top_n_collab=1, top_n_content=1):
    # Step 1: Get user-based collaborative filtering recommendations
    collab_recommendations = get_user_recipe_recommendations(user_id, user_ratings, user_sim_matrix, recipes_df, top_n=top_n_collab,random_n=0)

    # Step 2: Get content-based filtering recommendations
    content_recommendations = get_recipe_recommendations(recipe_title, cosine_sim_matrix, recipes_df, top_n=top_n_content)

    # Step 3: Combine the recommendations
    combined_recommendations = []

    for recommendation in collab_recommendations.itertuples():
        recipe_id = recommendation.id
        recipe_name = recommendation.name
        combined_recommendations.append((recipe_id, recipe_name))

    for recommendation in content_recommendations:
        recipe_id = recommendation[0]
        recipe_name = recommendation[1]
        combined_recommendations.append((recipe_id, recipe_name))

    return combined_recommendations


# Example usage
user_id = 4470
recipe_title = 'amish  tomato ketchup  for canning'
recommendations = get_combined_recommendations(user_id, recipe_title, user_ratings, user_sim, cosine_sim, recipes_df, top_n_collab=2, top_n_content=3)
print(f"Combined recommendations for user {user_id} and recipe '{recipe_title}':")
for i, (recipe_id, recipe_name) in enumerate(recommendations, 1):
    print(f"{i}: {recipe_name}")

Combined recommendations for user 4470 and recipe 'amish  tomato ketchup  for canning':
1: amish  tomato ketchup  for canning
2: love is in the air  beef fondue   sauces
3: momma s special  marinade
4: chef salad  dressing
5: fire  sauce
